In [1]:
import os
import sys
import json
import torch
import argparse
import numpy as np 

import model.model as module_arch
import utils.dataloader as module_data

from utils.metric import evaluate 
from utils.utils import decode_tags
from utils.metric import sequence_f1
from utils.parse_config import ConfigParser

resume = 'storage/checkpoints/debug_model/0806_234452_318276-based-lst20-lr_0.001-bs_16/model_best.pth'

args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str, help='config file path (default: None)')
args.add_argument('-r', '--resume', default=f"{resume}", type=str, help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default='cpu', type=str, help='indices of GPUs to enable (default: all)')
args.add_argument('--f', default='save', type=None)

# Check if in IPython environment
if any("ipykernel" in arg for arg in sys.argv):
    sys.argv = sys.argv[:1]  # Keep only the script name
    
config = ConfigParser.from_args(args)
logger = config.get_logger('test')

# setup dataloader instances
data_loader = config.init_obj('dataloader', module_data)

# build model architecturea
model = config.init_obj('arch', module_arch, num_tag=data_loader.num_tag, path_lm=data_loader.path_lm)

# get function handles of loss and metrics
metric_fns = {"sequence_f1": sequence_f1}

Train : 10 sentences
Dev : 10 sentences
Test : 7 sentences
Max sents length: 256 tokens
num_vocab: 25004
num_tag: 12
num_span: 5
num_spantag: 45


Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Load checkpoint
logger.info('Loading checkpoint: {} ...'.format(config.resume))
# checkpoint = torch.load(config.resume)
# state_dict = checkpoint['state_dict']
# model.load_state_dict(state_dict)

## 
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

INFO:test:Loading checkpoint: storage/checkpoints/debug_model/0806_234452_318276-based-lst20-lr_0.001-bs_16/model_best.pth ...


In [3]:
from utils.utils import predict, show

text = """
เมื่อวันที่29ก.ค.55 สำนักข่าวบีบีซีรายงานว่า มหาวิทยาลัยออกซ์ฟอร์ดของอังกฤษได้แก้ไขกฎการแต่งกายภายในมหาวิทยาลัย หลังชมรมเพื่อความหลากหลายทางเพศยื่นคำร้องว่ากฎที่มีอยู่เดิมไม่เป็นธรรมกับกลุ่มนศข้ามเพศtransgenderใน
มหาวิทยาลัย
"""
# Setup
lm_path = data_loader.path_lm
ids2tag = data_loader.ids2spantag
max_sent_length = data_loader.sent_length

tokens, out = predict(model, text, lm_path, ids2tag, max_sent_length)
tokens = [tk for tk in tokens if tk!=data_loader.pad]

print("|".join(tokens), "\n")
[show(x) for x in out];

<s>||เมื่อ|วันที่||29||ก|.|ค|.||55|||สํานักข่าว||บีบีซี||รายงาน||ว่า|||มหาวิทยาลั|ย||ออก|ซ์||ฟอร์ด||ของ||อังกฤษ||ได้||แก้ไข||กฎ|การ||แต่งกาย||ภายใน||มหาวิทยาลั|ย||หลัง||ชมรม|เพื่อ|ความ|หลากหลาย||ทางเพศ||ย|ื่นคําร้อง||ว่า||กฎ||ที่|มี||อยู่||เดิม|ไม่||เป็นธรรม||กับ|กลุ่ม||นศ||ข้าม||เพศ||trans|g|ender||ใน|||มหาวิทยาลั|ย||</s> 

[1, 2]         ORG            
[3, 4]         ORG            วันที่
[4, 5]         ORG            
[6, 7]         ORG            
[7, 8]         TTL            ก
[9, 10]        PER            ค
[13, 14]       ORG            
[14, 15]       ORG            
[16, 17]       ORG            
[17, 18]       NAME           บีบีซี
[18, 19]       ORG            
[22, 23]       ORG            
[23, 24]       ORG            
[25, 26]       PER            ย
[26, 27]       ORG            
[28, 29]       TTL            ซ์
[32, 33]       PER            ของ
[33, 34]       ORG            
[34, 35]       PER            อังกฤษ
[35, 36]       ORG            
[39, 40]       ORG         

'<pad>'